# Example: Compute the Primal and Dual Solutions of the Apple versus Oranges Problem
This example will familiarize students with the [linear programming dual problem](https://en.wikipedia.org/wiki/Dual_linear_program), and the relationship between the `primal` and `dual` solution to a linear program.

## Setup
This example requires several external libraries and a function to compute the outer product. Let's download and install these packages and call our `Include.jl` file.

In [3]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-4800-5800-Examples-Fall-2024/lecture/week-9/L9a`
    Updating `~/Desktop/julia_work/CHEME-4800-5800-Examples-Fall-2024/lecture/week-9/L9a/Project.toml`
  [a93c6f00] + DataFrames v1.7.0
  [60bf3e95] + GLPK v1.2.1
  [4076af6c] + JuMP v1.23.2
  [37e2e46d] ~ LinearAlgebra ⇒ v1.11.0
    Updating `~/Desktop/julia_work/CHEME-4800-5800-Examples-Fall-2024/lecture/week-9/L9a/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.5.0
  [523fee87] + CodecBzip2 v0.8.4
  [944b1d66] + CodecZlib v0.7.6
  [bbf7d656] + CommonSubexpressions v0.3.1
  [34da2185] + Compat v4.16.0
  [a8cc5b0e] + Crayons v4.1.1
  [9a962f9c] + DataAPI v1.16.0
  [a93c6f00] + DataFrames v1.7.0
  [864edb3b] + DataStructures v0.18.20
  [e2d170a0] + DataValueInterfaces v1.0.0
  [163ba53b] + DiffResults v1.1.0
  [b552c78f] + DiffRules v1.15.1
  [ffbed154] + DocStringExtensions v0.9.3
  [f6369f11] + ForwardDiff v0.10.36
  [60bf3e95] + GLPK v1.2.1
  [842dd82b] + InlineStrings v1.4.2
  [41ab

## Prerequisites
Let's set the prices, $\alpha_{i}$, and budget for our `Apple` versus `Oranges` problem. We'll store the prices in the `c` array and the coefficients in the utility function in the `α` variable.
* The $\alpha_{i}$ coefficients (because we have a linear utility function) are the [marginal utilities](https://en.wikipedia.org/wiki/Marginal_utility), i.e., they tell us the satisfaction we gain from consuming an additional unit of good $i$. They have units of `utils/qty`
* The $c_{i}$ coefficients represent the unit cost of each good, e.g., the cost of a single apple or orange. The $c_{i}$ coefficients have units of `USD/qty.`
* Finally, the `total_budget` variable holds the amount of money we spend on apples and oranges. The `total_budget` has units of `USD`.

In [5]:
# α = [0.55, 0.45]; # coefficients for case A (apples only)
α = [0.15, 0.55]; # coefficients for case B (oranges only)
# α = [2.0, 4.0]; # coefficients for case C (both)
c = [2.0 4.0]; # unit price of an Apple and an Orange
total_budget = 100.0; # total budget that we can spend

## Task 1: Compute the primal solution to the apple versus oranges problem
Let's solve the `primal` linear programming problem for the unknown values in our problem, i.e., the number of apples or oranges we should purchase to maximize our happiness function. The problem we are solving is a linear programming problem of the form:

$$
\begin{eqnarray*}
\text{maximize}~\mathcal{O}(\mathbf{x}) &=& U\left(x_{1},\dots,x_{n}\right) \\
\text{subject to}~\sum_{i\in{1,\dotsc,n}}c_{i}\cdot{x}_{i} & = & I\\
\text{and}~x_{i}&\geq&{0}\qquad{i=1,2,\dots,n}
\end{eqnarray*}
$$

The $c_{i}\geq{0}~\forall{i}$ denotes the cost of the object $i$, $x_{i}\geq{0}$ represents 
the amount of object $i$ purchased or consumed by the agent, and $I$ represent the budget that we have to spend. In this case, we'll use a __linear__ Utility function of the form:

$$
U(x) = \alpha_{1}\cdot{x}_{1}+\alpha_{2}\cdot{x}_{2}
$$

where $\alpha_{1}$ are the marginal utilities, $x_{1}$ denotes the quantity of `apples = 1`, and $x_{2}$ represents the number of `oranges = 2` that we purchase.

To solve this problem, let's first specify the bounds of the variables in the `bounds` variable. The first column of the array corresponds to the lower bound, while the second column corresponds to the upper bound for the unknown variable(s) $x_{i}$.

In [8]:
bounds = [
    0.0 100.0; # L U
    0.0 100.0; # L U
];

Next, we create an instance of the `MyLinearProgrammingProblemModel` model using a `build(...)` method and store this model in the `primal_problem` variable. 
* This model holds the data associated with the problem, e.g., the unit costs, the marginal utilities, the right-hand side vector, i.e., the budget, and the problem bounds

In [10]:
primal_problem = build(MyLinearProgrammingProblemModel, (
    
    c = α, # coefficients in Utility function
    A = c, # unit prices of x1 and x2 (we need this as a matrix in this formulation)
    b = [total_budget], # budge is the right-hand side
    
    # how much of x₁ and x₂ can be we buy?
    lb = bounds[:,1],
    ub = bounds[:,2]
));

Finally, we pass the `primal_problem` variable to the `solve(...)` method, which constructs the linear program using the [JuMP domain-specific language](https://jump.dev/). 
* The implementation of the `solve(...)` method is in the [src/Solver.jl](src/Solver.jl) file. It takes the data from the `primal_problem` instance, builds the various problem structures, and returns the solution in the `primal_solution` dictionary.

In [12]:
primal_solution = solve(primal_problem)

Dict{String, Any} with 2 entries:
  "argmax"          => [0.0, 25.0]
  "objective_value" => 13.75

## Task 2: Compute the dual solution to the apples versus oranges problem
The `primal` solution computed the combination of goods, e.g., apples and oranges, that maximized the utility (happiness) of the consumer subject to a budget constraint. The `dual` problem will minimize the expenditure required to achieve the maximum utility, i.e., spend the least amount of resources (money) to get the best outcome. The dual problem has the following structure:
$$
\begin{eqnarray*}
\text{minimize}~\mathcal{O}^{\prime}(\mathbf{y}) &=& I\cdot{y} \\
\text{subject to}~\mathbf{c}\cdot{y} & \geq & \mathbf{\alpha}\\
\text{and}~y&\geq&{0}
\end{eqnarray*}
$$
where $\mathbf{c}$ is the column-vector holding the unit costs, $\mathbf{\alpha}$ denotes the vector of marginal utilities and $y$ denotes the dual-decision variable.

To set up the `dual` problem, we create an instance of the `MyLinearProgrammingProblemModel` model using the `build(...)` method and store this model in the `dual_problem` variable.
* This model holds the data associated with the problem, e.g., the unit costs, the marginal utilities, the right-hand side vector, i.e., the budget, and the problem bounds (the same data as the `primal` problem)

In [15]:
dual_problem = build(MyLinearProgrammingDualProblemModel, (
    c = α,
    A = c,
    b = [total_budget],
    lb = bounds[:,1],
    ub = bounds[:,2]
));

Finally, we pass the `dual_problem` variable to the `solve(...)` method, which constructs the linear program using the [JuMP domain-specific language](https://jump.dev/). 
* The implementation of the `solve(...)` method is in the [src/Solver.jl](src/Solver.jl) file. It takes the data from the `dual_problem` instance, builds the various problem structures, and returns the solution in the `dual_solution` dictionary.

In [17]:
dual_solution = solve(dual_problem)

Dict{String, Any} with 2 entries:
  "argmin"          => [0.1375]
  "objective_value" => 13.75

### Hmmmm. How should we interpret the `dual` solution $y$?
One way to start thinking about the meaning of $y$ is to look at its units. From the units of $\alpha$, $c$ and the constraints on the dual problem, we can see that $y$ has units: `utils/USD.`
* Thus, the dual problem is minimizing the expenditure $I\cdot{y}$ (units: `USD`) such that we bound the marginal utility, i.e., find a marginal utility that is at least $\alpha_{\star}$, where $\star$ denotes the index of the good that is chosen (either apples or oranges in this case).

### Check: Does the `dual` solution say anything about the correct choice?
The `dual` solution does not explicitly calculate the number of apples or oranges we should purchase (the solution to the `primal` problem). Still, from our interpretation outlined above, it __should__ say something about which good we choose. 
* Let's check the relationship $y = \max\left\{\alpha_{1}/c_{1},\alpha_{2}/c_{2}\right\}$ and see which index is bigger (this should correspond to the largest scaled utility). Thus, the good that is chosen $i^{\star} = \arg\max\left\{\alpha_{1}/c_{1},\alpha_{2}/c_{2}\right\}$

In [20]:
test = [α[1]/c[1], α[2]/c[2]] |> argmax

2

### Check: Is the weak duality condition met?
Finally, we know from the [weak duality condition](https://en.wikipedia.org/wiki/Weak_duality) that $\mathcal{O}^{\prime}(\mathcal{y}^{\star})\geq\mathcal{O}(\mathcal{x}^{\star})$, where $\mathcal{y}^{\star}$ denotes the optimal solution of the dual problem, and $\mathcal{x}^{\star}$ denotes the optimal solution of the primal problem. 
* Let's check if this condition is true using the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert). If the Boolean `assertion fails,` then an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is thrown

In [22]:
@assert dual_solution["objective_value"] ≥ primal_solution["objective_value"]